In [53]:
import numpy as np
import pandas as pd

In [54]:
import pickle

def read_pickle(file):
    f = open(file, 'rb')
    l_d = pickle.load(f)
    return l_d

In [55]:
df_q_explor = pd.read_csv("/content/drive/MyDrive/data_colab/inex_data/query_explor2005_agr.tsv", sep='\t')
print(len(df_q_explor.index))

35


In [56]:
df_rel_explor_2005 = pd.read_csv("/content/drive/MyDrive/data_colab/inex_data/rel_explor2005_agr.tsv", sep='\t')
df_rel_explor_2005

,docid,qid,docno,tag,exhaustivity,specificity,gen_q,strict_q,intent_fin
0,ex/2003/x1055,202,ex/2003/x1055/article[1]/bdy[1]/sec[4],sec,2,0.57,1.14,0,exploratory
1,ex/2003/x1055,202,ex/2003/x1055/article[1]/bdy[1],bdy,2,0.32,0.64,0,exploratory
2,ex/2003/x1055,202,ex/2003/x1055/article[1],article,2,0.28,0.56,0,exploratory
3,ex/2003/x1055,202,ex/2003/x1055/article[1]/bdy[1]/sec[4]/ss1[3],ss,2,1.00,2.00,1,exploratory
4,ex/2003/x1055,202,ex/2003/x1055/article[1]/bdy[1]/sec[4]/ss1[4],ss,1,1.00,1.00,0,exploratory
...,...,...,...,...,...,...,...,...,...
17756,td/1996/l0321,286,td/1996/l0321/article[1]/bdy[1]/sec[2]/ip1[1],ip,1,1.00,1.00,0,exploratory
17757,td/1998/l0700,286,td/1998/l0700/article[1]/bdy[1]/sec[2],sec,1,0.11,0.11,0,exploratory
17758,td/1998/l0700,286,td/1998/l0700/article[1]/bdy[1],bdy,1,0.04,0.04,0,exploratory
17759,td/1998/l0700,286,td/1998/l0700/article[1],article,1,0.03,0.03,0,exploratory


In [57]:
df_rel_explor_2004 = pd.read_csv("/content/drive/MyDrive/data_colab/inex_data/rel_explor2004_agr.tsv", sep='\t')
df_rel_explor_2004

,docid,qid,docno,tag,exhaustivity,specificity,gen_q,sog_q,strict_q,intent_fin
0,mi/1995/m4019,127,mi/1995/m4019/article[1],article,3,1,0.75,0.25,0,exploratory
1,mi/1995/m4019,127,mi/1995/m4019/article[1]/bdy[1],bdy,3,1,0.75,0.25,0,exploratory
2,mi/1995/m4019,127,mi/1995/m4019/article[1]/bdy[1]/sec[3],sec,1,1,0.25,0.10,0,exploratory
3,mi/1995/m4019,127,mi/1995/m4019/article[1]/bdy[1]/sec[3]/fig[2],fig,1,3,0.50,0.75,0,exploratory
4,mi/1995/m4019,127,mi/1995/m4019/article[1]/bdy[1]/sec[3]/fig[2]/...,no,1,3,0.50,0.75,0,exploratory
...,...,...,...,...,...,...,...,...,...,...
16083,ts/2000/e1179,201,ts/2000/e1179/article[1],article,1,1,0.25,0.10,0,exploratory
16084,ts/2000/e1179,201,ts/2000/e1179/article[1]/bdy[1],bdy,1,1,0.25,0.10,0,exploratory
16085,ts/2000/e1179,201,ts/2000/e1179/article[1]/bdy[1]/sec[2],sec,1,1,0.25,0.10,0,exploratory
16086,ts/2000/e1179,201,ts/2000/e1179/article[1]/bdy[1]/sec[2]/ss1[1],ss,1,1,0.25,0.10,0,exploratory


In [58]:
df_rel_explor_2004 = df_rel_explor_2004[['qid','docno','sog_q']]
df_rel_explor_2004 = df_rel_explor_2004.rename(columns={'sog_q': 'label'})
df_rel_explor_2005 = df_rel_explor_2005.rename(columns={'gen_q': 'label'})
df_rel_explor_2005["qid"] = df_rel_explor_2005["qid"].astype(str)
df_rel_explor_2004["qid"] = df_rel_explor_2004["qid"].astype(str)

In [59]:
import numpy as np

def dcg_at_k(relevance_scores, k):
    """Calculate DCG for the given list of relevance scores up to position k."""
    relevance_scores = np.asfarray(relevance_scores)[:k]
    if relevance_scores.size:
        return np.sum(relevance_scores / np.log2(np.arange(2, relevance_scores.size + 2)))
    return 0

def ndcg_at_k(predicted_scores, ideal_scores, k):
    """Calculate NDCG for the given list of predicted and ideal relevance scores up to position k."""
    idcg = dcg_at_k(sorted(ideal_scores, reverse=True), k)
    if idcg == 0:
        return 0
    dcg = dcg_at_k(predicted_scores, k)
    return dcg / idcg

In [60]:
def ndcg_apply(df,k):
  grouped_result = df.groupby('qid')["label"].agg(list)
  l_ndcg = []
  for predicted_scores in grouped_result.to_list():
    ideal_scores = sorted(predicted_scores, reverse=True)
    ndcg_score = ndcg_at_k(predicted_scores, ideal_scores,k)
    #print(f"NDCG at 10: {ndcg_score:.4f}")
    l_ndcg.append(ndcg_score)
  return l_ndcg

BM25

In [61]:
bm25_res = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_explor_2005_features_and_sections.tsv', sep='\t')
bm25_res["qid"] = bm25_res["qid"].astype(str)


In [62]:
bm25_res = bm25_res.merge(df_rel_explor_2005[['qid', 'docno', 'label']], on=['qid', 'docno'], how='left')
bm25_res['label'] = bm25_res['label'].fillna(0)
bm25_res.head()

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf,document_length,tfidf_cosine_similarity,bert_cosine_similarity,query_in_text,label
0,202,37946,ex/2001/x3066/article[1]/bdy[1]/sec[2],0,18.867897,ontologies case study,exploratory,11.487210,APPROACH Companies offering photographic image...,8076.781991,4.229216,11.48721,5.844271,2.038815,159,0.212289,0.572751,0,0.00
1,202,37951,ex/2001/x3066/article[1]/bdy[1]/sec[7],1,18.416160,ontologies case study,exploratory,11.176510,CONCLUSION study scratched surface regarding p...,8076.781991,4.229216,11.48721,5.844271,2.038815,522,0.117569,0.543533,0,0.00
2,202,37947,ex/2001/x3066/article[1]/bdy[1]/sec[3],2,18.332671,ontologies case study,exploratory,11.173441,DEVELOPING ONTOLOGIES define semantic annotati...,8076.781991,4.229216,11.48721,5.844271,2.038815,937,0.062268,0.574287,0,1.22
3,202,51210,tk/2003/k0442/article[1]/bdy[1]/sec[6],3,18.081575,ontologies case study,exploratory,11.043574,Cross-Ontology Evaluations studies addressed q...,8076.781991,4.229216,11.48721,5.844271,2.038815,1586,0.165072,0.631319,0,0.00
4,202,37599,ex/2001/x1026/article[1]/bdy[1]/sec[4],4,17.877747,ontologies case study,exploratory,10.257502,"KNOWLEDGE METAPROCESS decade, CommonKADS gener...",8076.781991,4.229216,11.48721,5.844271,2.038815,493,0.195070,0.540878,0,0.00


In [63]:
mean_sr_bm25 = np.mean(ndcg_apply(bm25_res,100))
print("NDCG@100 soft rank non intent aware:", mean_sr_bm25)

NDCG@100 soft rank non intent aware: 0.3890578910797464


1. INTENT-AGNOSTIC

In [64]:
train_na = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_explor_2004_features_and_sections.tsv', sep='\t')
train_na["qid"] = train_na["qid"].astype(str)


In [65]:
train_na = train_na.merge(df_rel_explor_2004[['qid', 'docno', 'label']], on=['qid', 'docno'], how='left')
train_na['label'] = train_na['label'].fillna(0)
train_na.head()

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf,document_length,tfidf_cosine_similarity,bert_cosine_similarity,query_in_text,label
0,127,4740,mi/1995/m4019/article[1]/bdy[1]/sec[6],0,58.768128,godel lukasiewicz other fuzzy implication defi...,exploratory,32.335857,Future design actual RFP implementation assume...,7690.482804,3.449013,32.335857,5.253062,3.037769,100,0.196721,0.358958,0,0.00
1,127,4739,mi/1995/m4019/article[1]/bdy[1]/sec[5],1,37.435215,godel lukasiewicz other fuzzy implication defi...,exploratory,20.616273,Hardware implementation Godel Implication (ope...,7690.482804,3.449013,32.335857,5.253062,3.037769,1192,0.084546,0.377896,0,0.25
2,127,35405,tk/1999/k0153/article[1]/bdy[1]/sec[4],2,25.728032,godel lukasiewicz other fuzzy implication defi...,exploratory,14.253312,FUZZY IF-THEN RULES Among techniques developed...,7690.482804,3.449013,32.335857,5.253062,3.037769,1892,0.501975,0.520324,0,1.00
3,127,26586,ex/1995/x2004/article[1]/bdy[1]/sec[2],3,24.189762,godel lukasiewicz other fuzzy implication defi...,exploratory,13.696832,"Elkan's fallacy: missed midpoint Briefly, hypo...",7690.482804,3.449013,32.335857,5.253062,3.037769,210,0.104810,0.454137,0,0.00
4,127,35817,tk/1998/k0551/article[1]/bdy[1]/sec[3],4,24.062681,godel lukasiewicz other fuzzy implication defi...,exploratory,13.625252,"SOFT FUNCTIONAL REQUIREMENTS section, propose ...",7690.482804,3.449013,32.335857,5.253062,3.037769,267,0.060399,0.192803,0,0.10


In [66]:
test_na = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_explor_2005_features_and_sections.tsv',sep = '\t')
test_na["qid"] = test_na["qid"].astype(str)

In [67]:
test_na = test_na.merge(df_rel_explor_2005[['qid', 'docno', 'label']], on=['qid', 'docno'], how='left')
test_na['label'] = test_na['label'].fillna(0)
test_na.head()

,qid,docid,docno,rank,score_bm25,query,intent_fin,score_tfidf,text,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf,document_length,tfidf_cosine_similarity,bert_cosine_similarity,query_in_text,label
0,202,37946,ex/2001/x3066/article[1]/bdy[1]/sec[2],0,18.867897,ontologies case study,exploratory,11.487210,APPROACH Companies offering photographic image...,8076.781991,4.229216,11.48721,5.844271,2.038815,159,0.212289,0.572751,0,0.00
1,202,37951,ex/2001/x3066/article[1]/bdy[1]/sec[7],1,18.416160,ontologies case study,exploratory,11.176510,CONCLUSION study scratched surface regarding p...,8076.781991,4.229216,11.48721,5.844271,2.038815,522,0.117569,0.543533,0,0.00
2,202,37947,ex/2001/x3066/article[1]/bdy[1]/sec[3],2,18.332671,ontologies case study,exploratory,11.173441,DEVELOPING ONTOLOGIES define semantic annotati...,8076.781991,4.229216,11.48721,5.844271,2.038815,937,0.062268,0.574287,0,1.22
3,202,51210,tk/2003/k0442/article[1]/bdy[1]/sec[6],3,18.081575,ontologies case study,exploratory,11.043574,Cross-Ontology Evaluations studies addressed q...,8076.781991,4.229216,11.48721,5.844271,2.038815,1586,0.165072,0.631319,0,0.00
4,202,37599,ex/2001/x1026/article[1]/bdy[1]/sec[4],4,17.877747,ontologies case study,exploratory,10.257502,"KNOWLEDGE METAPROCESS decade, CommonKADS gener...",8076.781991,4.229216,11.48721,5.844271,2.038815,493,0.195070,0.540878,0,0.00


In [68]:
relevance_labels = train_na['label'].to_list(
)
print(relevance_labels)
# Remove duplicates by converting the list to a set, then convert back to a sorted list
label_gain = sorted(set(relevance_labels))
print(label_gain)

int_labels = [label_gain.index(label) for label in relevance_labels]

[0.0, 0.25, 1.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [69]:
relevance_labels_test = np.array(test_na['label'].to_list())

# Define the number of bins for label mapping (0 to 6 corresponds to 7 gains)
n_bins = 7

# Use numpy to digitize the labels into bins
int_labels_test = np.digitize(relevance_labels_test, np.linspace(min(relevance_labels_test), max(relevance_labels_test), n_bins))

# Adjust to ensure labels are within 0 to 6
int_labels_test = int_labels_test - 1  # Digitize returns 1-indexed bins, so subtract 1


In [70]:
import numpy as np

def dcg_at_k(relevance_scores, k):
    """Calculate DCG for the given list of relevance scores up to position k."""
    relevance_scores = np.asfarray(relevance_scores)[:k]
    if relevance_scores.size:
        return np.sum(relevance_scores / np.log2(np.arange(2, relevance_scores.size + 2)))
    return 0

def ndcg_at_k(predicted_scores, ideal_scores, k):
    """Calculate NDCG for the given list of predicted and ideal relevance scores up to position k."""
    idcg = dcg_at_k(sorted(ideal_scores, reverse=True), k)
    if idcg == 0:
        return 0
    dcg = dcg_at_k(predicted_scores, k)
    return dcg / idcg


In [71]:
def ndcg_apply(df,k):
  grouped_result = df.groupby('qid')["label"].agg(list)
  l_ndcg = []
  for predicted_scores in grouped_result.to_list():
    ideal_scores = sorted(predicted_scores, reverse=True)
    ndcg_score = ndcg_at_k(predicted_scores, ideal_scores,k)
    #print(f"NDCG at 10: {ndcg_score:.4f}")
    l_ndcg.append(ndcg_score)
  return l_ndcg

In [72]:
!pip install pandas lightgbm

In [73]:
import lightgbm as lgb

In [74]:
#Prepare the features (X) and labels (y)

X_train_na = train_na[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values  # Including the user intent feature
y_train_na = train_na['label'].values  # Use the scaled labels
print(len(X_train_na))

# Extract the group sizes (number of documents per query) for training
group_train_na = train_na.groupby('qid').size().to_numpy()

# Create the LightGBM dataset
train_data_na = lgb.Dataset(X_train_na, label=int_labels, group=group_train_na)
print(len(X_train_na))
print(len(relevance_labels))
l_tfidf = train_na['var_tfidf'].to_list()

35951
35951
35951


In [75]:
# Prepare the features (X) and labels (y) for test data
X_test_na = test_na[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values
y_test_na = test_na['label'].values  # Use the actual labels

# Extract the group sizes (number of documents per query) for test data
group_test_na = test_na.groupby('qid').size().to_numpy()

# Create LightGBM dataset for test set
test_data_na = lgb.Dataset(X_test_na, label=int_labels_test, group=group_test_na)


In [76]:
#min_data_in_leaf previousely 50, num leaves 16 max depth 6

params_lm = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting': 'gbdt',
    'num_leaves': 16,
    'max_depth': 6,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'min_data_in_leaf': 100,  # Increase this to regularize more
    'label_gain': label_gain,  # Increase this to regularize more
    'lambda_l1': 0.1,  # Add L1 regularization
    'lambda_l2': 0.1,  # Add L2 regularization
}



In [77]:
import matplotlib.pyplot as plt

#Initialize dictionaries to store results
evals_result = {'train': [], 'test': []}

# Custom callback to track evaluation results manually and print every 10 rounds
def record_evaluation(env):
    results = env.evaluation_result_list
    train_ndcg = results[0][2]  # Training NDCG (assuming first is train)
    test_ndcg = results[1][2]   # Test NDCG (assuming second is test)

    # Append to the evals_result dict
    evals_result['train'].append(train_ndcg)
    evals_result['test'].append(test_ndcg)

    # Print every 10 rounds
    if (env.iteration + 1) % 10 == 0:
        print(f"Round {env.iteration + 1}: Train NDCG = {train_ndcg}, Test NDCG = {test_ndcg}")

# Train the model and track NDCG on training and test sets
ranker_lm_na = lgb.train(
    params_lm,
    train_data_na,
    num_boost_round=150,
    valid_sets=[train_data_na, test_data_na],  # Include both training and test sets for evaluation
    valid_names=['train', 'test'],  # Give names to the datasets
    callbacks=[record_evaluation, lgb.early_stopping(stopping_rounds=30)]
)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044542 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 907
[LightGBM] [Info] Number of data points in the train set: 35951, number of used features: 9
Training until validation scores don't improve for 30 rounds
Round 10: Train NDCG = 0.5779835390946502, Test NDCG = 0.545861266934777
Round 20: Train NDCG = 0.6497942386831276, Test NDCG = 0.5743726867915524
Round 30: Train NDCG = 0.6683127572016461, Test NDCG = 0.5799961117355862
Round 40: Train NDCG = 0.6683127572016461, Test NDCG = 0.5799961117355862
Round 50: Train NDCG = 0.6683127572016461, Test NDCG = 0.5799961117355862
Early stopping, best iteration is:
[22]	train's ndcg@1: 0.668313	train's ndcg@2: 0.601488	train's ndcg@3: 0.597959	train's ndcg@4: 0.577908	train's ndcg@5: 0.565338	test's ndcg@1: 0.35254	test's ndcg@2: 0.341539	test's ndcg@3: 0.346941	test's ndcg@4: 0.33796	test's ndcg@5: 0.328709


In [78]:
def apply_lambdamart_na(df, model):
    ranked_results = []
    queries = df['qid'].unique()
    for qid in queries:
        subset = df[df['qid'] == qid]
        features = subset[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values
        scores = model.predict(features)
        subset = subset.assign(lambdamart_score=scores)
        ranked_results.append(subset.sort_values(by='lambdamart_score', ascending=False))

    return pd.concat(ranked_results)


In [79]:
result_lm_na = apply_lambdamart_na(test_na, ranker_lm_na)

In [80]:
result_lm_na = result_lm_na.reset_index(drop=True)
result_lm_na["qid"] = result_lm_na["qid"].astype(str)

In [81]:
mean_lm_na = np.mean(ndcg_apply(result_lm_na,20))
print("NDCG@100 lambdamart non intent aware:", mean_lm_na)

NDCG@100 lambdamart non intent aware: 0.372788004749007


2.INTENT-AWARE

In [82]:
train_ia = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_explor_2004_features_sections_RM3.tsv', sep='\t')
train_ia["qid"] = train_ia["qid"].astype(str)


In [83]:
train_ia= train_ia.merge(df_rel_explor_2004[['qid', 'docno', 'label']], on=['qid', 'docno'], how='left')
train_ia['label'] = train_ia['label'].fillna(0)
train_ia.head()

,qid,docid,docno,rank,score_bm25,query_0,query,score_tfidf,text,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf,document_length,tfidf_cosine_similarity,bert_cosine_similarity,query_in_text,label
0,127,4740,mi/1995/m4019/article[1]/bdy[1]/sec[6],0,75.632037,godel lukasiewicz other fuzzy implication defi...,godel rfp fuzzi neuron triangular implic lukas...,32.335857,Future design actual RFP implementation assume...,7069.303204,3.445117,32.335857,5.421245,3.139733,100,0.192373,0.358958,0,0.00
1,127,4739,mi/1995/m4019/article[1]/bdy[1]/sec[5],1,44.607482,godel lukasiewicz other fuzzy implication defi...,godel rfp fuzzi neuron triangular implic lukas...,20.616273,Hardware implementation Godel Implication (ope...,7069.303204,3.445117,32.335857,5.421245,3.139733,1192,0.082348,0.377896,0,0.25
2,127,4374,mi/1995/m4006/article[1]/bdy[1]/sec[1],2,25.366469,godel lukasiewicz other fuzzy implication defi...,godel rfp fuzzi neuron triangular implic lukas...,7.171018,Fuzzy hardware developments major vehicle popu...,7069.303204,3.445117,32.335857,5.421245,3.139733,442,0.263021,0.401855,0,0.00
3,127,4738,mi/1995/m4019/article[1]/bdy[1]/sec[4],3,24.836361,godel lukasiewicz other fuzzy implication defi...,godel rfp fuzzi neuron triangular implic lukas...,5.807560,General network design generic network consist...,7069.303204,3.445117,32.335857,5.421245,3.139733,653,0.040830,0.262411,0,0.00
4,127,35405,tk/1999/k0153/article[1]/bdy[1]/sec[4],4,22.687070,godel lukasiewicz other fuzzy implication defi...,godel rfp fuzzi neuron triangular implic lukas...,14.253312,FUZZY IF-THEN RULES Among techniques developed...,7069.303204,3.445117,32.335857,5.421245,3.139733,1892,0.497921,0.520324,0,1.00


In [84]:
test_ia = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_explor_2005_features_sections_RM3.tsv', sep='\t')
test_ia["qid"] = test_ia["qid"].astype(str)

In [85]:
test_ia = test_ia.merge(df_rel_explor_2005[['qid', 'docno', 'label']], on=['qid', 'docno'], how='left')
test_ia['label'] = test_ia['label'].fillna(0)
test_ia.head()


,qid,docid,docno,rank,score_bm25,query_0,query,score_tfidf,text,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf,document_length,tfidf_cosine_similarity,bert_cosine_similarity,query_in_text,label
0,202,37946,ex/2001/x3066/article[1]/bdy[1]/sec[2],0,28.626876,ontologies case study,case photograph subject editor photo ontolog c...,11.487210,APPROACH Companies offering photographic image...,6988.958534,4.229216,11.48721,6.136048,1.985902,159,0.213656,0.572751,0,0.00
1,202,37951,ex/2001/x3066/article[1]/bdy[1]/sec[7],1,27.012216,ontologies case study,case photograph subject editor photo ontolog c...,11.176510,CONCLUSION study scratched surface regarding p...,6988.958534,4.229216,11.48721,6.136048,1.985902,522,0.118830,0.543533,0,0.00
2,202,37947,ex/2001/x3066/article[1]/bdy[1]/sec[3],2,26.258647,ontologies case study,case photograph subject editor photo ontolog c...,11.173441,DEVELOPING ONTOLOGIES define semantic annotati...,6988.958534,4.229216,11.48721,6.136048,1.985902,937,0.062881,0.574287,0,1.22
3,202,37948,ex/2001/x3066/article[1]/bdy[1]/sec[4],3,22.500993,ontologies case study,case photograph subject editor photo ontolog c...,6.954463,ANNOTATING PHOTOGRAPHS USING MULTIMEDIA INFORM...,6988.958534,4.229216,11.48721,6.136048,1.985902,298,0.000000,0.280342,0,0.00
4,202,41197,ex/2002/x1054/article[1]/bdy[1]/sec[2],4,21.624783,ontologies case study,case photograph subject editor photo ontolog c...,8.551419,ONTOLOGY LANGUAGES Several ontology languages ...,6988.958534,4.229216,11.48721,6.136048,1.985902,533,0.059484,0.628480,0,0.00


In [86]:
relevance_labels_ia = train_ia['label'].to_list(
)
print(relevance_labels_ia)
# Remove duplicates by converting the list to a set, then convert back to a sorted list
label_gain_ia = sorted(set(relevance_labels_ia))
print(label_gain_ia)

int_labels_ia = [label_gain_ia.index(label) for label in relevance_labels_ia]
print(int_labels_ia)

[0.0, 0.25, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [87]:
relevance_labels_ia_test = np.array(test_ia['label'].to_list())

# Define the number of bins for label mapping (0 to 6 corresponds to 7 gains)
n_bins = 7

# Use numpy to digitize the labels into bins
int_labels_ia_test = np.digitize(relevance_labels_ia_test, np.linspace(min(relevance_labels_ia_test), max(relevance_labels_ia_test), n_bins))

# Adjust to ensure labels are within 0 to 6
int_labels_ia_test = int_labels_ia_test - 1  # Digitize returns 1-indexed bins, so subtract 1

# Check the new integer labels
print(relevance_labels_ia_test)
print(int_labels_ia_test)

[0.   0.   1.22 ... 0.   0.   0.  ]
[0 0 3 ... 0 0 0]


In [88]:
# Prepare the features (X) and labels (y)
X_train_ia = train_ia[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','bert_cosine_similarity','tfidf_cosine_similarity','query_in_text']].values  # Including the user intent feature
y_train_ia = train_ia['label'].values  # Use the scaled labels
print(len(X_train_ia))

# Extract the group sizes (number of documents per query) for training
group_train_ia = train_ia.groupby('qid').size().to_numpy()

# Create the LightGBM dataset
train_data_ia = lgb.Dataset(X_train_ia, label=int_labels_ia, group=group_train_ia)
print(len(X_train_ia))
print(len(relevance_labels_ia))


27499
27499
27499


In [89]:
# Prepare the features (X) and labels (y) for test data
X_test_ia = test_ia[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','bert_cosine_similarity','tfidf_cosine_similarity','query_in_text']].values
y_test_ia = test_ia['label'].values  # Use the actual labels

# Extract the group sizes (number of documents per query) for test data
group_test_ia = test_ia.groupby('qid').size().to_numpy()

# Create LightGBM dataset for test set
test_data_ia = lgb.Dataset(X_test_ia, label=int_labels_ia_test, group=group_test_ia)
print(len(X_test_ia))
print(len(relevance_labels_ia_test))

38825
38825


In [90]:
#Initialize dictionaries to store results
evals_result = {'train': [], 'test': []}

# Custom callback to track evaluation results manually and print every 10 rounds
def record_evaluation(env):
    results = env.evaluation_result_list
    train_ndcg = results[0][2]  # Training NDCG (assuming first is train)
    test_ndcg = results[1][2]   # Test NDCG (assuming second is test)

    # Append to the evals_result dict
    evals_result['train'].append(train_ndcg)
    evals_result['test'].append(test_ndcg)

    # Print every 10 rounds
    if (env.iteration + 1) % 10 == 0:
        print(f"Round {env.iteration + 1}: Train NDCG = {train_ndcg}, Test NDCG = {test_ndcg}")


# Train the model with early stopping using callbacks
ranker_lm_ia = lgb.train(
    params_lm,
    train_data_ia,
    num_boost_round=150,  # Set maximum number of boosting rounds
    valid_sets=[train_data_ia, test_data_ia],  # Include both training and test sets for evaluation
    valid_names=['train', 'test'],
    callbacks=[record_evaluation, lgb.early_stopping(stopping_rounds=15)]
)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003156 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1162
[LightGBM] [Info] Number of data points in the train set: 27499, number of used features: 10
Training until validation scores don't improve for 15 rounds
Round 10: Train NDCG = 0.6205761316872428, Test NDCG = 0.60106085588546
Round 20: Train NDCG = 0.668724279835391, Test NDCG = 0.6248516050595815
Round 30: Train NDCG = 0.668724279835391, Test NDCG = 0.6484926099461368
Early stopping, best iteration is:
[19]	train's ndcg@1: 0.668724	train's ndcg@2: 0.639179	train's ndcg@3: 0.613665	train's ndcg@4: 0.614259	train's ndcg@5: 0.610336	test's ndcg@1: 0.383333	test's ndcg@2: 0.362915	test's ndcg@3: 0.348578	test's ndcg@4: 0.342092	test's ndcg@5: 0.337757


In [91]:
def apply_lambdamart_ia(df, model):
    ranked_results = []
    queries = df['qid'].unique()
    for qid in queries:
        subset = df[df['qid'] == qid]
        features = subset[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','bert_cosine_similarity','tfidf_cosine_similarity','query_in_text']].values
        scores = model.predict(features)
        subset = subset.assign(lambdamart_score=scores)
        ranked_results.append(subset.sort_values(by='lambdamart_score', ascending=False))

    return pd.concat(ranked_results)

In [92]:
result_lm_ia = apply_lambdamart_ia(test_ia, ranker_lm_ia)
result_lm_ia = result_lm_ia.reset_index(drop=True)
result_lm_ia["qid"] = result_lm_ia["qid"].astype(str)

In [93]:
mean_sr_bm25 = round(np.mean(ndcg_apply(bm25_res,100)),4)
print("NDCG@100 BM25:", mean_sr_bm25)
mean_sr_na = round(np.mean(ndcg_apply(result_lm_na,100)),4)
print("NDCG@100 lambdamart non intent aware:", mean_sr_na)
mean_sr_ia = round(np.mean(ndcg_apply(result_lm_ia,100)),4)
print("NDCG@100 lambdamart intent aware:", mean_sr_ia)

NDCG@100 BM25: 0.3891
NDCG@100 lambdamart non intent aware: 0.4183
NDCG@100 lambdamart intent aware: 0.4574


In [94]:
improv = (np.mean(ndcg_apply(result_lm_na,100)) - np.mean(ndcg_apply(bm25_res,100)))/np.mean(ndcg_apply(bm25_res,100))*100
print(f"improvement na is {round(improv,2)} percent")

improvement na is 7.52 percent


In [95]:
improv = (np.mean(ndcg_apply(result_lm_ia,100)) - np.mean(ndcg_apply(bm25_res,100)))/np.mean(ndcg_apply(bm25_res,100))*100
print(f"improvement ia is {round(improv,2)} percent")

improvement ia is 17.57 percent


In [96]:
# Define the parameters for Soft rank
params_sr = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting': 'gbdt',
    'num_leaves': 16,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'label_gain': label_gain  # Adjust this to cover the range of your scaled labels
}

In [97]:
import matplotlib.pyplot as plt


#Initialize dictionaries to store results
evals_result = {'train': [], 'test': []}

# Custom callback to track evaluation results manually and print every 10 rounds
def record_evaluation(env):
    results = env.evaluation_result_list
    train_ndcg = results[0][2]  # Training NDCG (assuming first is train)
    test_ndcg = results[1][2]   # Test NDCG (assuming second is test)

    # Append to the evals_result dict
    evals_result['train'].append(train_ndcg)
    evals_result['test'].append(test_ndcg)

    # Print every 10 rounds
    if (env.iteration + 1) % 10 == 0:
        print(f"Round {env.iteration + 1}: Train NDCG = {train_ndcg}, Test NDCG = {test_ndcg}")

# Train the model and track NDCG on training and test sets
ranker_sr_na = lgb.train(
    params_sr,
    train_data_na,
    num_boost_round=150,
    valid_sets=[train_data_na, test_data_na],  # Include both training and test sets for evaluation
    valid_names=['train', 'test'],  # Give names to the datasets
    callbacks=[record_evaluation, lgb.early_stopping(stopping_rounds=20)]
)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001048 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 907
[LightGBM] [Info] Number of data points in the train set: 35951, number of used features: 9
Training until validation scores don't improve for 20 rounds
Round 10: Train NDCG = 0.5905349794238683, Test NDCG = 0.5372804214747615
Round 20: Train NDCG = 0.6183127572016461, Test NDCG = 0.5679237756024249
Round 30: Train NDCG = 0.6942386831275721, Test NDCG = 0.5851053863556982
Early stopping, best iteration is:
[14]	train's ndcg@1: 0.594239	train's ndcg@2: 0.530238	train's ndcg@3: 0.574868	train's ndcg@4: 0.555428	train's ndcg@5: 0.543423	test's ndcg@1: 0.282222	test's ndcg@2: 0.254887	test's ndcg@3: 0.262748	test's ndcg@4: 0.269678	test's ndcg@5: 0.267313


In [98]:
def apply_soft_rank_na(df):
  qid_list = df['qid'].unique()
  ranked_results = []
  for qid in qid_list:
      subset = df[df['qid'] == qid]
      features = subset[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','document_length','query_in_text']].values  # Only numeric features for prediction
      scores = ranker_sr_na.predict(features)
      subset = subset.assign(soft_rank_score=scores)
      ranked_results.append(subset.sort_values(by='soft_rank_score', ascending=False))
  return pd.concat(ranked_results)

# Apply the AdaRank model to re-rank the documents
result_sr_na = apply_soft_rank_na(test_na)

# Reset index to get a clean DataFrame
result_sr_na = result_sr_na.reset_index(drop=True)

In [99]:
result_sr_na["qid"] = result_sr_na["qid"].astype(str)

In [100]:
#Initialize dictionaries to store results
evals_result = {'train': [], 'test': []}

# Custom callback to track evaluation results manually and print every 10 rounds
def record_evaluation(env):
    results = env.evaluation_result_list
    train_ndcg = results[0][2]  # Training NDCG (assuming first is train)
    test_ndcg = results[1][2]   # Test NDCG (assuming second is test)

    # Append to the evals_result dict
    evals_result['train'].append(train_ndcg)
    evals_result['test'].append(test_ndcg)

    # Print every 10 rounds
    if (env.iteration + 1) % 10 == 0:
        print(f"Round {env.iteration + 1}: Train NDCG = {train_ndcg}, Test NDCG = {test_ndcg}")


# Train the model with early stopping using callbacks
ranker_sr_ia = lgb.train(
    params_sr,
    train_data_ia,
    num_boost_round=150,  # Set maximum number of boosting rounds
    valid_sets=[train_data_ia, test_data_ia],  # Include both training and test sets for evaluation
    valid_names=['train', 'test'],
    callbacks=[record_evaluation, lgb.early_stopping(stopping_rounds=15)]
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000960 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1162
[LightGBM] [Info] Number of data points in the train set: 27499, number of used features: 10
Training until validation scores don't improve for 15 rounds
Round 10: Train NDCG = 0.6590534979423869, Test NDCG = 0.6281969324770285
Round 20: Train NDCG = 0.6720164609053498, Test NDCG = 0.6405925460042049
Round 30: Train NDCG = 0.6909465020576131, Test NDCG = 0.6470254902972455
Early stopping, best iteration is:
[19]	train's ndcg@1: 0.647942	train's ndcg@2: 0.635145	train's ndcg@3: 0.642192	train's ndcg@4: 0.628665	train's ndcg@5: 0.635787	test's ndcg@1: 0.381111	test's ndcg@2: 0.327059	test's ndcg@3: 0.346502	test's ndcg@4: 0.346308	test's ndcg@5: 0.338021


In [101]:
def apply_soft_rank_ia(df):
  qid_list = df['qid'].unique()
  ranked_results = []
  for qid in qid_list:
      subset = df[df['qid'] == qid]
      features = subset[['score_bm25','score_tfidf','sum_tfidf','mean_tfidf','min_tfidf','max_tfidf','var_tfidf','bert_cosine_similarity','tfidf_cosine_similarity','query_in_text']].values  # Only numeric features for prediction
      scores = ranker_sr_ia.predict(features)
      subset = subset.assign(soft_rank_score=scores)
      ranked_results.append(subset.sort_values(by='soft_rank_score', ascending=False))
  return pd.concat(ranked_results)

In [102]:
result_sr_ia = apply_soft_rank_ia(test_ia)
result_sr_ia = result_sr_ia.reset_index(drop=True)
result_sr_ia["qid"] = result_sr_ia["qid"].astype(str)

In [103]:
mean_sr_bm25 = round(np.mean(ndcg_apply(bm25_res,100)),4)
print("NDCG@100 BM25:", mean_sr_bm25)
mean_sr_na = round(np.mean(ndcg_apply(result_sr_na,100)),4)
print("NDCG@100 soft rank non intent aware:", mean_sr_na)
mean_sr_ia = round(np.mean(ndcg_apply(result_sr_ia,100)),4)
print("NDCG@100 soft rank intent aware:", mean_sr_ia)

NDCG@100 BM25: 0.3891
NDCG@100 soft rank non intent aware: 0.4094
NDCG@100 soft rank intent aware: 0.4592


In [104]:
from scipy.stats import ttest_rel

nb_el = 100

baseline = ndcg_apply(bm25_res,nb_el)
ndcg_lm_na = ndcg_apply(result_lm_na,nb_el)
ndcg_lm_ia = ndcg_apply(result_lm_ia,nb_el)
ndcg_sr_na = ndcg_apply(result_sr_na,nb_el)
ndcg_sr_ia = ndcg_apply(result_sr_ia,nb_el)




# Perform t-tests
t_stat1, p_val1 = ttest_rel(baseline, ndcg_lm_na)
t_stat2, p_val2 = ttest_rel(baseline, ndcg_lm_ia)
t_stat3, p_val3 = ttest_rel(baseline, ndcg_sr_na)
t_stat4, p_val4 = ttest_rel(baseline, ndcg_sr_ia)

# Collect p-values
p_values = [p_val1, p_val2,p_val3,p_val4]
print(p_values)

[0.17394892528091985, 0.00023389805189713485, 0.21715631938357416, 3.904874068234164e-05]


In [105]:
# Number of tests
num_tests = len(p_values)
print(num_tests)

# Apply Bonferroni correction
bonferroni_corrected_pvals = [min(p * num_tests, 1.0) for p in p_values]
print(bonferroni_corrected_pvals)

4
[0.6957957011236794, 0.0009355922075885394, 0.8686252775342966, 0.00015619496272936657]


In [106]:
alpha = 0.05

# Apply Bonferroni correction
bonferroni_corrected_pvals = [min(p * num_tests, 1.0) for p in p_values]
l_names = ['Lambdamart na','Lambdamart_ia','Softrank_na','Softrank_ia']

# Check for significance and print results
for i, (t_stat, p_val, corrected_pval) in enumerate(zip([t_stat1,t_stat2,t_stat3,t_stat4], p_values, bonferroni_corrected_pvals)):
    is_significant = corrected_pval <= alpha
    significance_str = "Significant" if is_significant else "Not Significant"
    print(f"Result {i+1}:")
    print(l_names[i])
    print(f"  t-statistic = {t_stat:.4f}")
    print(f"  original p-value = {p_val:.4f}")
    print(f"  Bonferroni corrected p-value = {corrected_pval:.4f} ({significance_str})")
    print()

Result 1:
Lambdamart na
  t-statistic = -1.3887
  original p-value = 0.1739
  Bonferroni corrected p-value = 0.6958 (Not Significant)

Result 2:
Lambdamart_ia
  t-statistic = -4.1131
  original p-value = 0.0002
  Bonferroni corrected p-value = 0.0009 (Significant)

Result 3:
Softrank_na
  t-statistic = -1.2575
  original p-value = 0.2172
  Bonferroni corrected p-value = 0.8686 (Not Significant)

Result 4:
Softrank_ia
  t-statistic = -4.7240
  original p-value = 0.0000
  Bonferroni corrected p-value = 0.0002 (Significant)

